# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [156]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def get_cities_area(city):
    # Some names need to be updated manually
    di_cities = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return di_cities.get(city,city)

def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa']: c = c + ' ' + city.split()[1]
    if c == "New Jersey": c = 'New York'
    if c == "San Jose": c = 'San Francisco'
    if c == "Vegas": c = 'Las Vegas'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    return c

def nhl_correlation(): 
    cities = pd.read_html("assets/wikipedia_data.html",na_values='—')[1] # '-' replaced by NaN
    cities = cities.iloc[:-1,[0,3,5,6,7,8]] # remove last row 'Total', extract only 6 relevant cols
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True) # data cleaning
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.astype({'Population': 'int64'})
    cities['area'] = cities['Metropolitan area'].apply(get_cities_area)
    cities.drop(['NFL','MLB','NBA','Metropolitan area','NHL'], inplace=True, axis=1)
    
    nhl_df = pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df.loc[nhl_df['year'] == 2018] # extract data where year = 2018 only
    nhl_df = nhl_df[['team','W','L']] # extract team, no. of Wins and Lose cols
    nhl_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
    divisionIndexes = nhl_df[nhl_df['team'].str.contains("Division")].index # get indexes where team contains 'Division'
    nhl_df.drop(divisionIndexes , inplace=True) # drop rows of those indexes
    nhl_df = nhl_df.astype({'W': 'int32','L': 'int32' }) # cast to integer for calculating ratios
    nhl_df['ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L']) # calculate ratio
    nhl_df['area'] = nhl_df['team'].apply(get_area)
    nhl_df.drop(['W','L'], inplace=True, axis=1)
    
    df = pd.merge(cities, nhl_df, how='inner')
    df = df.dropna().groupby(['area']).agg({"Population":np.mean,"ratio":np.mean})
    population_by_region = df['Population']
    win_loss_by_region = df['ratio']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

(0.012486162921209907, 0.9497182859911791)

In [111]:
# Part 1 - cities
def get_cities_area(city):
    # Some names need to be updated manually
    di_cities = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return di_cities.get(city,city)

cities = pd.read_html("assets/wikipedia_data.html",na_values='—')[1] # '-' replaced by NaN
cities = cities.iloc[:-1,[0,3,5,6,7,8]] # remove last row 'Total', extract only 6 relevant cols
cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True) # data cleaning
cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
cities = cities.astype({'Population': 'int64'})
cities['area'] = cities['Metropolitan area'].apply(get_cities_area)
cities.drop(['NFL','MLB','NBA','Metropolitan area','NHL'], inplace=True, axis=1)
cities

,Population,area
0,20153634,New York
1,13310447,Los Angeles
2,6657982,San Francisco
3,9512999,Chicago
4,7233323,Dallas
5,6131977,Washington
6,6070500,Philadelphia
7,4794447,Boston
8,3551036,Minneapolis
9,2853077,Denver


In [123]:
# Part 2 - nhl_df
def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa']: c = c + ' ' + city.split()[1]
    if c == "New Jersey": c = 'New York'
    if c == "San Jose": c = 'San Francisco'
    if c == "Vegas": c = 'Las Vegas'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    return c

nhl_df = pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df.loc[nhl_df['year'] == 2018] # extract data where year = 2018 only
nhl_df = nhl_df[['team','W','L']] # extract team, no. of Wins and Lose cols
nhl_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
divisionIndexes = nhl_df[nhl_df['team'].str.contains("Division")].index # get indexes where team contains 'Division'
nhl_df.drop(divisionIndexes , inplace=True) # drop rows of those indexes
nhl_df = nhl_df.astype({'W': 'int32','L': 'int32' }) # cast to integer for calculating ratios
nhl_df['ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L']) # calculate ratio
nhl_df['area'] = nhl_df['team'].apply(get_area)
nhl_df.drop(['W','L'], inplace=True, axis=1)

nhl_df

,team,ratio,area
1,Tampa Bay Lightning,0.701299,Tampa Bay
2,Boston Bruins,0.714286,Boston
3,Toronto Maple Leafs,0.653333,Toronto
4,Florida Panthers,0.594595,Miami
5,Detroit Red Wings,0.434783,Detroit
6,Montreal Canadiens,0.420290,Montreal
7,Ottawa Senators,0.394366,Ottawa
8,Buffalo Sabres,0.357143,Buffalo
10,Washington Capitals,0.653333,Washington
11,Pittsburgh Penguins,0.618421,Pittsburgh


In [110]:
# Part 3 - merge the 2 dfs, groupby the area, agg population and win/loss ratio
df = pd.merge(cities, nhl_df, how='inner')
df = df.dropna().groupby(['area']).agg({"Population":np.mean,"ratio":np.mean})
population_by_region = df['Population']
win_loss_by_region = df['ratio']

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [157]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def get_cities_area(city):
    # Some names need to be updated manually
    di_cities = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return di_cities.get(city,city)

def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa','Oklahoma']: c = c + ' ' + city.split()[1]
    if c == "Vegas": c = 'Las Vegas'
    if c in ["New Jersey", 'Brooklyn']: c = 'New York'
    if c in ["San Jose", 'Golden']: c = 'San Francisco'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    if c == "Utah": c = 'Salt Lake City'
    if c == "Indiana": c = 'Indianapolis'
    return c

def nba_correlation():
    cities = pd.read_html("assets/wikipedia_data.html",na_values='—')[1] # '-' replaced by NaN
    cities = cities.iloc[:-1,[0,3,5,6,7,8]] # remove last row 'Total', extract only 6 relevant cols
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True) # data cleaning
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.astype({'Population': 'int64'})
    cities['area'] = cities['Metropolitan area'].apply(get_cities_area)
    cities.drop(['NFL','MLB','NBA','Metropolitan area','NHL'], inplace=True, axis=1)
   
    nba_df = pd.read_csv("assets/nba.csv")
    nba_df = nba_df.loc[nba_df['year'] == 2018] # extract data where year = 2018 only
    nba_df = nba_df[['team','W/L%']] # extract team, ratios
    nba_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
    nba_df['team'].replace('\(.*\)','',regex=True, inplace=True) # remove (1) in the team name
    nba_df.rename(columns={'W/L%':'ratio'}, inplace=True)
    nba_df = nba_df.astype({'ratio': 'float64'})
    nba_df['area'] = nba_df['team'].apply(get_area)
    
    df = pd.merge(cities, nba_df, how='outer')
    df = df.dropna().groupby(['area']).agg({"Population":np.mean,"ratio":np.mean})
    population_by_region = df['Population']
    win_loss_by_region = df['ratio']

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17636350642182938

In [153]:
# NBA ver
def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa','Oklahoma']: c = c + ' ' + city.split()[1]
    if c == "Vegas": c = 'Las Vegas'
    if c in ["New Jersey", 'Brooklyn']: c = 'New York'
    if c in ["San Jose", 'Golden']: c = 'San Francisco'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    if c == "Utah": c = 'Salt Lake City'
    if c == "Indiana": c = 'Indianapolis'
    return c

nba_df = pd.read_csv("assets/nba.csv")
nba_df = nba_df.loc[nba_df['year'] == 2018] # extract data where year = 2018 only
nba_df = nba_df[['team','W/L%']] # extract team, ratios
nba_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
nba_df['team'].replace('\(.*\)','',regex=True, inplace=True) # remove (1) in the team name
nba_df.rename(columns={'W/L%':'ratio'}, inplace=True)
nba_df = nba_df.astype({'ratio': 'float64'})
nba_df['area'] = nba_df['team'].apply(get_area)

nba_df

,team,ratio,area
0,Toronto Raptors,0.720,Toronto
1,Boston Celtics,0.671,Boston
2,Philadelphia 76ers,0.634,Philadelphia
3,Cleveland Cavaliers,0.610,Cleveland
4,Indiana Pacers,0.585,Indianapolis
5,Miami Heat,0.537,Miami
6,Milwaukee Bucks,0.537,Milwaukee
7,Washington Wizards,0.524,Washington
8,Detroit Pistons,0.476,Detroit
9,Charlotte Hornets,0.439,Charlotte


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [158]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def get_cities_area(city):
    # Some names need to be updated manually
    di_cities = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return di_cities.get(city,city)

def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa','Oklahoma','Kansas']: c = c + ' ' + city.split()[1]
    if c == "Vegas": c = 'Las Vegas'
    if c in ["New Jersey", 'Brooklyn']: c = 'New York'
    if c in ["San Jose", 'Golden', 'Oakland']: c = 'San Francisco'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    if c == "Utah": c = 'Salt Lake City'
    if c == "Indiana": c = 'Indianapolis'
    if c == "Texas": c = 'Dallas'
    return c

def mlb_correlation(): 
    cities = pd.read_html("assets/wikipedia_data.html",na_values='—')[1] # '-' replaced by NaN
    cities = cities.iloc[:-1,[0,3,5,6,7,8]] # remove last row 'Total', extract only 6 relevant cols
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True) # data cleaning
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.astype({'Population': 'int64'})
    cities['area'] = cities['Metropolitan area'].apply(get_cities_area)
    cities.drop(['NFL','MLB','NBA','Metropolitan area','NHL'], inplace=True, axis=1)
   
    mlb_df = pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df.loc[mlb_df['year'] == 2018] # extract data where year = 2018 only
    mlb_df = mlb_df[['team','W-L%']] # extract team, ratios
    mlb_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
    mlb_df['team'].replace('\(.*\)','',regex=True, inplace=True) # remove (1) in the team name
    mlb_df.rename(columns={'W-L%':'ratio'}, inplace=True)
    mlb_df = mlb_df.astype({'ratio': 'float64'})
    mlb_df['area'] = mlb_df['team'].apply(get_area)
    
    df = pd.merge(cities, mlb_df, how='outer')
    df = df.dropna().groupby(['area']).agg({"Population":np.mean,"ratio":np.mean})
    population_by_region = df['Population']
    win_loss_by_region = df['ratio']
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15003737475409495

In [151]:
# my attempt
def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa','Oklahoma','Kansas']: c = c + ' ' + city.split()[1]
    if c == "Vegas": c = 'Las Vegas'
    if c in ["New Jersey", 'Brooklyn']: c = 'New York'
    if c in ["San Jose", 'Golden', 'Oakland']: c = 'San Francisco'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    if c == "Utah": c = 'Salt Lake City'
    if c == "Indiana": c = 'Indianapolis'
    if c == "Texas": c = 'Dallas'
    return c

mlb_df = pd.read_csv("assets/mlb.csv")
mlb_df = mlb_df.loc[mlb_df['year'] == 2018] # extract data where year = 2018 only
mlb_df = mlb_df[['team','W-L%']] # extract team, ratios
mlb_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
mlb_df['team'].replace('\(.*\)','',regex=True, inplace=True) # remove (1) in the team name
mlb_df.rename(columns={'W-L%':'ratio'}, inplace=True)
mlb_df = mlb_df.astype({'ratio': 'float64'})
mlb_df['area'] = mlb_df['team'].apply(get_area)
mlb_df

,team,ratio,area
0,Boston Red Sox,0.667,Boston
1,New York Yankees,0.617,New York
2,Tampa Bay Rays,0.556,Tampa Bay
3,Toronto Blue Jays,0.451,Toronto
4,Baltimore Orioles,0.290,Baltimore
5,Cleveland Indians,0.562,Cleveland
6,Minnesota Twins,0.481,Minneapolis
7,Detroit Tigers,0.395,Detroit
8,Chicago White Sox,0.383,Chicago
9,Kansas City Royals,0.358,Kansas City


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [160]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def get_cities_area(city):
    # Some names need to be updated manually
    di_cities = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return di_cities.get(city,city)

def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa','Oklahoma','Kansas', 'Green']: c = c + ' ' + city.split()[1]
    if c == "Vegas": c = 'Las Vegas'
    if c in ["New Jersey", 'Brooklyn']: c = 'New York'
    if c in ["San Jose", 'Golden', 'Oakland']: c = 'San Francisco'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    if c == "Utah": c = 'Salt Lake City'
    if c == "Indiana": c = 'Indianapolis'
    if c == "Texas": c = 'Dallas'
    if c == "New England": c = 'Boston'
    if c == "Tennessee": c = 'Nashville'
    return c

def nfl_correlation(): 
    cities = pd.read_html("assets/wikipedia_data.html",na_values='—')[1] # '-' replaced by NaN
    cities = cities.iloc[:-1,[0,3,5,6,7,8]] # remove last row 'Total', extract only 6 relevant cols
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True) # data cleaning
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.astype({'Population': 'int64'})
    cities['area'] = cities['Metropolitan area'].apply(get_cities_area)
    cities.drop(['NFL','MLB','NBA','Metropolitan area','NHL'], inplace=True, axis=1)
   
    nfl_df = pd.read_csv("assets/nfl.csv")
    nfl_df = nfl_df.loc[nfl_df['year'] == 2018] # extract data where year = 2018 only
    nfl_df = nfl_df[['team','W-L%']] # extract team, ratios
    nfl_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
    divisionIndexes = nfl_df[nfl_df['team'].str.contains("FC")].index # get indexes where team contains 'Division'
    nfl_df.drop(divisionIndexes , inplace=True) # drop rows of those indexes
    nfl_df.rename(columns={'W-L%':'ratio'}, inplace=True)
    nfl_df = nfl_df.astype({'ratio': 'float64'})
    nfl_df['area'] = nfl_df['team'].apply(get_area)
    
    df = pd.merge(cities, nfl_df, how='inner')
    df = df.dropna().groupby(['area']).agg({"Population":np.mean,"ratio":np.mean})
    population_by_region = df['Population']
    win_loss_by_region = df['ratio']
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.008330914787036972

In [152]:
# my attempt
def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa','Oklahoma','Kansas', 'Green']: c = c + ' ' + city.split()[1]
    if c == "Vegas": c = 'Las Vegas'
    if c in ["New Jersey", 'Brooklyn']: c = 'New York'
    if c in ["San Jose", 'Golden', 'Oakland']: c = 'San Francisco'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    if c == "Utah": c = 'Salt Lake City'
    if c == "Indiana": c = 'Indianapolis'
    if c == "Texas": c = 'Dallas'
    if c == "New England": c = 'Boston'
    if c == "Tennessee": c = 'Nashville'
    return c

nfl_df = pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df.loc[nfl_df['year'] == 2018] # extract data where year = 2018 only
nfl_df = nfl_df[['team','W-L%']] # extract team, ratios
nfl_df['team'].replace('\*','',regex=True, inplace=True) # remove * in the team name
divisionIndexes = nfl_df[nfl_df['team'].str.contains("FC")].index # get indexes where team contains 'Division'
nfl_df.drop(divisionIndexes , inplace=True) # drop rows of those indexes
nfl_df.rename(columns={'W-L%':'ratio'}, inplace=True)
nfl_df = nfl_df.astype({'ratio': 'float64'})
nfl_df['area'] = nfl_df['team'].apply(get_area)
nfl_df

,team,ratio,area
1,New England Patriots,0.688,Boston
2,Miami Dolphins,0.438,Miami
3,Buffalo Bills,0.375,Buffalo
4,New York Jets,0.250,New York
6,Baltimore Ravens,0.625,Baltimore
7,Pittsburgh Steelers,0.594,Pittsburgh
8,Cleveland Browns,0.469,Cleveland
9,Cincinnati Bengals,0.375,Cincinnati
11,Houston Texans,0.688,Houston
12,Indianapolis Colts+,0.625,Indianapolis


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [155]:
def get_area(city):
    c = city.split()[0]
    if c in ['New','Los','St.','San','Tampa','Oklahoma','Kansas', 'Green']: c = c + ' ' + city.split()[1]
    if c == "Vegas": c = 'Las Vegas'
    if c in ["New Jersey", 'Brooklyn']: c = 'New York'
    if c in ["San Jose", 'Golden', 'Oakland']: c = 'San Francisco'
    if c == "Arizona": c = 'Phoenix'
    if c == "Florida": c = 'Miami'
    if c == "Anaheim": c = 'Los Angeles'
    if c == "Colorado": c = 'Denver'
    if c == "Carolina": c = 'Raleigh'
    if c == "Minnesota": c = 'Minneapolis'
    if c == "Utah": c = 'Salt Lake City'
    if c == "Indiana": c = 'Indianapolis'
    if c == "Texas": c = 'Dallas'
    if c == "New England": c = 'Boston'
    if c == "Tennessee": c = 'Nashville'
    return c

def get_cities_area(city):
    di_cities = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return di_cities.get(city,city)

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")

cities = pd.read_html("assets/wikipedia_data.html",na_values='—')[1] # '-' replaced by NaN
cities = cities.iloc[:-1,[0,3,5,6,7,8]] # remove last row 'Total', extract only 6 relevant cols
cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True) # data cleaning
cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
cities = cities.astype({'Population': 'int64'})
cities['area'] = cities['Metropolitan area'].apply(get_cities_area)
# cities.drop(['NFL','MLB','NBA','Metropolitan area','NHL'], inplace=True, axis=1)

# Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
# df.loc["NFL","NFL"] should return np.nan
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,NaN,NaN,NaN
NBA,NaN,NaN,NaN,NaN
NHL,NaN,NaN,NaN,NaN
MLB,NaN,NaN,NaN,NaN
